In [1]:
import numpy as np
from scipy.spatial.distance import cdist
import random
import sklearn.neighbors as neighbors

In [2]:
# X a DxN array, d the dimension of the subspace
#Returns mu = 1xD average of the data,
# U = Dxd low dimensional basis,
# Y = dxN low dimensional coordinates
def Pca(X, d):
    avg = np.mean(X, axis = 1)
    centered = X - avg.reshape(X.shape[0], 1)
    U, S, Vt = np.linalg.svd(centered, full_matrices=False)
    Y = np.diag(S[0:d]) @ Vt[0:d, :]
    return avg, U[:, 0:d], Y
    

In [6]:
#Returns the number of dimensions spanning the low dimensional subspace of the data
#Parameters: X, DxN data matrix; tau, eigenvalue threshold (as a percentage of total energy)
def PcaModelSelection(X, tau):
    U, S, Vt = np.linalg.svd(X, full_matrices = False)
    tot = 0
    for i in S:
        tot += i**2
    count = 0
    while (tot > tau):
        tot -= S[count]**2
        count = count + 1
    return count

In [44]:
# Data to test dimensional agreements
# a = np.random.rand(50, 100)
# b = np.zeros((50, 70))
# b[0, :] = np.transpose(np.ones(70)) * 50
# b = b + np.random.rand(50,70)
# c = np.concatenate((a, b), axis = 1)

In [88]:
#Soft thresholding operator
#Inputs: x, a float, and tau, the threshold value
#Returns: Soft threshold of x
def STau(x, tau):
    if (x < -tau):
        return x + tau
    elif (x > tau):
        return x - tau
    else:
        return 0

In [31]:
#Singular value thresholding of matrix A
#Inputs: A, matrix to be thresholded and tau, the threshold value
#Returns U*STau(Sigma, tau)*Vt, where U*Sigma*Vt is the SVD of A
def DTau(A, tau):
    U, S, Vt = np.linalg.svd(A, full_matrices=False)
    for i in S:
        i = STau(i, tau)
    A = U @ np.diag(S) @ Vt
    return A

In [32]:
#Low rank matrix completion using convex optimization
#and Singular Value Thresholding
#Inputs: X- The observed data matrix (DxN)
#W- matrix of 0's and 1's corresponding to unknown and known entries (DxN)
#tau- parameter in potimization problem
#beta- step size in dual gradient ascent
def LRMC(X, W, tau, beta):
    Z0 = np.zeros(X.size())
    A = Z0
    Z1 = Z0 + beta * (W * (X - A))
    threshold = np.norm(Z1, 1) * 0.00001
    while (np.norm(Z1-Z0, 1) > threshold):
        Z0 = Z1
        A = DTau(W * Z1, tau)
        Z1 = Z0 + beta * (W * (X - A))
    return A

In [47]:
#Returns the kernel matrix given data and a specified kernel function
#Inputs: X- DxN data matrix, kernel- kernel function
#Returns: matrix k where k[i][j] is the kernel function output
#with the i-th and j-th data columns supplied as inputs
def getKernel(X, kernel):
    N = X.shape[1]
    k = np.empty([N, N])
    for i in range(N):
        for j in range(N):
            k[i, j] = kernel(X[:, i], X[:, j])
    return k

#Returns the local PCA coordinates from a nonlinear embedding
#in high dimensional space
#Inputs: X- DxN data matrix, kernel-kernel function, d- number of principal components to compute
#Returns: Y- dxN local coordinate matrix
def Kpca(X, kernel, d):
    K = getKernel(X, kernel)
    N = X.shape[1]
    one_N = np.ones(N)
    J = np.identity(N) - (1 / N) * (one_N @ np.transpose(one_N))
    k_hat = J @ K @ J
    w, v = np.linalg.eig(k_hat)
    print(w.shape)
    Y = np.diag(np.sqrt(w[0:d])) @ np.transpose(v)[0:d, :]
    return Y

In [48]:
def PolynomialKernel(x, y):
    return np.dot(x, y)

In [72]:
#Matrix completion by direct power factorization. More efficient than LRMC
#calculating SVD at every iteration.
#Inputs: X- DxN data matrix with missing entries, d- the number of principal components to use
#Outputs: Completion of X
def MCByPower(X, d):
    Y0 = np.random.rand(d, X.shape[1])
    Y_next = Y0
    Y = np.zeros(Y_next.shape) 
    U_next = X @ Y0 @ np.linalg.inv(Y0 @ np.transpose(Y0))
    Q, R = np.linalg.qr(U_next)
    U_next = Q
    U = np.zeros(U_next.shape)
    threshold = np.norm(U_next @ Y_next) * 0.001
    while (np.linalg.norm(U_next @ Y_next) - np.norm(U @ Y) > threshold):
        Y = Y_next
        Y_next = np.transpose(U) @ X
        U = U_next
        U_next = X @ X @ Y_next @ np.linalg.inv(Y_next @ np.transpose(Y_next))
        Q, R = np.linalg.qr(U_next)
        U_next = Q
    return U_next @ Y_next

In [104]:
#Implementation of k-means algorithm
#Inputs: X- DxN data matrix, d- number of groups
#Returns: Dxn- matrix where the i-th column is the i-th centroid
#W- nxN membership matrix W where W[i, j] = 1 iff X_j is a member of gorup i
def KMeans(X, n):
    N = X.shape[1]

    start_indices = random.sample(range(0, N), n)
    centers = X[:, start_indices]
    w1 = np.zeros((n, N))
    w0 = np.ones((n, N))
    count = 0
    while (np.linalg.norm(w1 - w0, 'nuc') > 0):
        w0 = w1
        w1 = np.zeros((n, N))
        dist = cdist(np.transpose(X), np.transpose(centers), 'euclidean')
        groups = np.argmin(dist, axis=1)
        print(groups)
        count += 1
        w1[groups, range(N)] = 1
        centers = X @ (w1.T / np.sum(w1, axis = 1))
    return centers, w1

In [105]:
#Implementation of spectral clustering algorithm
#Inputs: n- number of groups, W- a NxN affinity matrix
#Returns: nxN membership matrix where W[i, j] = 1 iff X_j is a
#member of gorup i, computed by k-means
def SpectralCluster(n, W):
    N = W.shape[0]
    D = np.diag(W @ np.ones(N))
    L = D - W
    w, v = np.linalg.eig(L)
    Y = w[:, -n].T
    centers, groups = KMeans(Y, n)
    return groupsvision

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


(array([[50.43136763,  0.49555876],
        [ 0.47141483,  0.54761687],
        [ 0.48160644,  0.48673812],
        [ 0.52366881,  0.52917423],
        [ 0.45799294,  0.55437026],
        [ 0.55243638,  0.54765783],
        [ 0.50688354,  0.51519311],
        [ 0.46756168,  0.48262751],
        [ 0.50002283,  0.52491398],
        [ 0.53905569,  0.49179028],
        [ 0.42949098,  0.47727026],
        [ 0.49119132,  0.49720717],
        [ 0.55940495,  0.53157426],
        [ 0.51658544,  0.51775062],
        [ 0.47931969,  0.51087333],
        [ 0.51908833,  0.50813499],
        [ 0.49862152,  0.50153215],
        [ 0.462986  ,  0.48858178],
        [ 0.57388241,  0.49311133],
        [ 0.49487335,  0.47998455],
        [ 0.51833069,  0.4698341 ],
        [ 0.56402453,  0.49095644],
        [ 0.4659353 ,  0.472994  ],
        [ 0.47890512,  0.48843033],
        [ 0.50074262,  0.5049674 ],
        [ 0.50841531,  0.52651755],
        [ 0.46705961,  0.50494076],
        [ 0.56158523,  0.448

In [18]:
def GetNNByRadius(X, epsilon):
    tree = neighbors.BallTree(X.T)
    ind = tree.query_radius(X.T[:], r = epsilon)
    return ind

In [ ]:
def GetNNByNumber(X, K):
    tree = neighbors.BallTree(X.T)
    dist, ind = tree.query(X[:], k = K)
    return ind

In [ ]:
def STauMatrix(X, tau):
    A = np.zeros(X.shape)
    for idx, x in np.ndenumerate(X):
        A[idx] = STau(x, tau)
    return A

In [ ]:
#Creates a sparse local representation C so that X = XC by minimizing L1 norm and 
#augmenting reconstruction error
#Inputs: X- DxN data Matrix, tau- reconstruction error parameter (higher means that the matrix
# will be more exact at the expense of sparsity), mu- second optimization parameter of the 
#augmented Lagrangian
#Returns: sparse representation C s.t. X is approx. XC
def LASSOMinimizationADMM(X, tau, mu):
    D, N = X.shape
    C0 = np.zeros([N, N])
    L0 = np.zeros([N, N])
    Z1 = np.linalg.inv(tau * X.T @ X + mu * np.identity(N)) * (tau * X.T @ X)
    C1 = STauMatrix(Z1, 1 / mu).fill_diagonal(0)
    L1 = mu * (Z1 - C1)
    threshold = 0.0001 * np.linalg.norm(C1, 1)
    while (np.linalg.norm(C1 - C0, 1) > threshold):
        C0 = C1
        L0 = L1
        Z0 = Z1
        Z1 = np.linalg.inv(tau * X.T @ X + mu * np.identity(N)) * \
            (tau * X.T @ X + mu * (C0 - L0 / mu))
        C1 = STauMatrix(Z1 + L0 / mu, 1 / mu).fill_diagonal(0)
        L1 = L0 + mu * (Z1 - C1)
    return C

In [ ]:
#Computes clustering of data into subspaces with the sparse subspace clustering
#algorithm augmented to handle noisy data.
#Inputs: X- DxN data matrix, n- number of subspaces, tau- Optimization parameter
#Increasing tau will prioritize an accurate reconstruction at the risk that the
#underlying representation is not sparse, leading to 
#subspace mixing. Tau too low will not give an accurate
#reconstruction of the data and therefore clustering cannot be trusted.
#Returns: nxN membership matrix W where W[i, j] = 1 iff X_j is a
#member of gorup i
def SSCWithNoise(X, n, tau):
    C = LASSOMinimizationADMM(X, tau, tau)
    W = C + C.T
    return SpectralCluster(n, W)

In [ ]:
#Performs the local subspace affinity algorithm using euclidean distance
#Inputs: X- DxN data matrix, K- number of nearest neighbors to query (should 
#select K to be sure it's higher than the dimensions of the subspaces), n-
#number of subspaces to cluster
#Returns: nxN membership matrix W where W[i, j] = 1 iff X_j is a
#member of gorup i
def LSA(X, K, n):
    N, D = X.shape
    ind = GetNNByNumber(X, K)
    subspaces = {}
    for i in range(N):
        NN = X[:, ind[i]]
        d = PcaModelSelection(NN, 0.99)
        mu, U, Y = Pca(NN, d)
        subspaces[i] = [U, d]
    W = np.zeros([N, N])
    for i in range(N):
        for j in range(i, N):
            U, S, Vt = np.linalg.svd(subspaces[i][0].T @ subspaces[j][0])
            w = np.square(S[0:np.minimum(subspaces[i][1], \
                                               subspaces[j][1])]).sum
            W[i, j] = w
            W[j, i] = w
    return SpectralCluster(W, n)